In [ ]:
### 예스 24 크롤링 

# webdriver-manager 라이브러리 불러오기
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

In [ ]:
from selenium import webdriver
browser = webdriver.Chrome()       # python file 로 변환 후 실행

In [ ]:
## yes24 베스트셀러 크롤링 

url = 'https://www.yes24.com/Product/category/bestseller?CategoryNumber=001&sumgb=06'
browser.get(url)

In [ ]:
import time

link_list = []    # 링크 수집을 위한 빈 리스트

for i in range(1,4):
    print("*" * 10, f'현재 {i}페이지 수집중입니다.', "*" * 10)

    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
    
    time.sleep(3)

print(link_list)

In [ ]:
len(link_list)

In [ ]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 DB에 저장 (csv)
# SQL 워크벤치 불러온 뒤 테이블 생성

# Terminal에서 pip install pymysql
import pymysql
import time
import re
from datetime import datetime

conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'class-password',
    db = 'yes24',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
)


with conn.cursor() as cur:
    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # OperationalError: (1292, "Incorrect date value: '2024년 09월 25일' for column 'publishing' at row 1")
        # 2024년 9월 25일 -> 2024-09-25 날짜 격식 수정
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)

        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year),int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2024-01-01"

        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text

        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text

        if review == "예약판매" or "첫번째 리뷰어가 되어주세요.":
            review = "0"
        else:
            review = int(review.replace(",", ""))  # "," 콤마 삭제

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",", ""))  # "," 콤마 삭제

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",", ""))  # "," 콤마 삭제

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        print('full_text :', full_text) # 베스트만 들어옴
        parts = full_text.split(" | ")

        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part))   # 숫자만 추출
            except:
                ranking = 0

            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))   # 숫자만 추출
            except:
                ranking_weeks = 0


        # 위 try 문구로 아래 두줄 대체
        # ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[1][:-1]
        # ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]
        

        sql = """INSERT INTO Books(
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES(
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """
    
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))